In [12]:
import pandas as pd
import pandas as pd
import numpy as np
import random
import os
from sklearn.model_selection import train_test_split

In [23]:
SEED = 42 
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

In [14]:
dir_path = "../data/"
# Cargar el archivo limpio
df = pd.read_csv(f"{dir_path}clean_data_log_structured.csv")
print(set(df["source"]))
# Verificar estructura y primeras filas
print(df.shape)
print(df.columns)
df.head()

{'BGL', 'Zookeeper', 'Apache', 'Hadoop', 'Android'}
(9410, 6)
Index(['Content', 'Level', 'datetime', 'content_length', 'source',
       'Clean_Content'],
      dtype='object')


,Content,Level,datetime,content_length,source,Clean_Content
0,printFreezingDisplayLogsopening app wtoken = A...,Informativo,2024-03-17 16:13:38.811,22,Android,printfreezingdisplaylogsopening app wtoken app...
1,"acquire lock=233570404, flags=0x1, tag=""View L...",Informativo,2024-03-17 16:13:38.819,13,Android,acquire lock flags tag view lock com android s...
2,"ready=true,policy=3,wakefulness=1,wksummary=0x...",Informativo,2024-03-17 16:13:38.820,17,Android,ready true policy wakefulness wksummary uasumm...
3,Skipping AppWindowToken{df0798e token=Token{78...,Informativo,2024-03-17 16:13:38.839,17,Android,skipping appwindowtoken token token activityre...
4,visible is system.time.showampm,Informativo,2024-03-17 16:13:38.859,3,Android,visible time showampm


In [15]:
import pandas as pd

train_list = []
test_list = []

for source in df['source'].unique():
    df_source = df[df['source'] == source]

    # Calcular la cantidad mínima de elementos en cada categoría (clase minoritaria)
    level_counts = df_source['Level'].value_counts()
    min_class = level_counts.idxmin()
    min_count = level_counts[min_class]

    # Calcular cuántos elementos extraer para test desde la clase minoritaria
    test_n = max(int(min_count * 0.25), 1)

    # Diccionario para guardar cuántos tomar de cada categoría
    test_samples_per_class = {}

    for category in df_source['Level'].unique():
        df_cat = df_source[df_source['Level'] == category]

        # Mezclar aleatoriamente
        df_cat_shuffled = df_cat.sample(frac=1, random_state=42)

        # Usar el mismo número de muestras (test_n) que se tomó de la clase minoritaria
        extract_n = min(test_n, len(df_cat_shuffled))  # evita errores si alguna clase tiene < test_n

        # Dividir
        test_split = df_cat_shuffled.iloc[:extract_n]
        train_split = df_cat_shuffled.iloc[extract_n:]

        test_list.append(test_split)
        train_list.append(train_split)

# Unir los splits
train_data = pd.concat(train_list).reset_index(drop=True)
external_test_data = pd.concat(test_list).reset_index(drop=True)


## Train data

In [16]:
print("Train data:", train_data.shape)

Train data: (9090, 6)


In [17]:
print(train_data['Level'].value_counts())

Level
Informativo    6038
Advertencia    2253
Error           799
Name: count, dtype: int64


In [18]:
# Agrupar por fuente
grouped = train_data.groupby('source')

# Recorrer cada grupo y calcular estadísticas
for name, group in grouped:
    print(f"\n Dataset: {name}")
    print(f"- Total registros: {len(group)}")
    print("- Distribución por nivel:")
    print(group['Level'].value_counts())
    print(f"- Longitud promedio del mensaje: {group['content_length'].mean():.2f} palabras")
    print(f"- Periodo: desde {group['datetime'].min()} hasta {group['datetime'].max()}")



 Dataset: Android
- Total registros: 1984
- Distribución por nivel:
Level
Informativo    1816
Advertencia     166
Error             2
Name: count, dtype: int64
- Longitud promedio del mensaje: 7.27 palabras
- Periodo: desde 2024-03-17 16:13:38.811 hasta 2024-03-17 16:16:09.141

 Dataset: Apache
- Total registros: 1273
- Distribución por nivel:
Level
Informativo    989
Error          284
Name: count, dtype: int64
- Longitud promedio del mensaje: 3.82 palabras
- Periodo: desde 2005-12-04 04:47:44.000 hasta 2005-12-05 19:15:57.000

 Dataset: BGL
- Total registros: 1989
- Distribución por nivel:
Level
Informativo    1594
Error           389
Advertencia       6
Name: count, dtype: int64
- Longitud promedio del mensaje: 4.94 palabras
- Periodo: desde 2005-06-03 22:42:50 hasta 2006-01-03 15:13:09

 Dataset: Hadoop
- Total registros: 1854
- Distribución por nivel:
Level
Informativo    973
Advertencia    767
Error          114
Name: count, dtype: int64
- Longitud promedio del mensaje: 6.45 pal

## External test data

In [19]:
print("External test data:", external_test_data.shape)

External test data: (320, 6)


In [20]:
print(external_test_data['Level'].value_counts())

Level
Informativo    138
Error          138
Advertencia     44
Name: count, dtype: int64


In [21]:
# Agrupar por fuente
grouped = external_test_data.groupby('source')

# Recorrer cada grupo y calcular estadísticas
for name, group in grouped:
    print(f"\n Dataset: {name}")
    print(f"- Total registros: {len(group)}")
    print("- Distribución por nivel:")
    print(group['Level'].value_counts())
    print(f"- Longitud promedio del mensaje: {group['content_length'].mean():.2f} palabras")
    print(f"- Periodo: desde {group['datetime'].min()} hasta {group['datetime'].max()}")



 Dataset: Android
- Total registros: 3
- Distribución por nivel:
Level
Informativo    1
Advertencia    1
Error          1
Name: count, dtype: int64
- Longitud promedio del mensaje: 4.33 palabras
- Periodo: desde 2024-03-17 16:13:46.764 hasta 2024-03-17 16:15:58.884

 Dataset: Apache
- Total registros: 188
- Distribución por nivel:
Level
Informativo    94
Error          94
Name: count, dtype: int64
- Longitud promedio del mensaje: 3.98 palabras
- Periodo: desde 2005-12-04 04:47:44.000 hasta 2005-12-05 19:14:11.000

 Dataset: BGL
- Total registros: 6
- Distribución por nivel:
Level
Informativo    2
Error          2
Advertencia    2
Name: count, dtype: int64
- Longitud promedio del mensaje: 5.17 palabras
- Periodo: desde 2005-06-12 00:54:54 hasta 2005-09-20 18:57:30

 Dataset: Hadoop
- Total registros: 114
- Distribución por nivel:
Level
Informativo    38
Error          38
Advertencia    38
Name: count, dtype: int64
- Longitud promedio del mensaje: 6.10 palabras
- Periodo: desde 2015-10-

## Save datasets

In [22]:
dir_path = "../process/"
train_data.to_csv(f"{dir_path}train_data.csv", index=False)
external_test_data.to_csv(f"{dir_path}external_test_data.csv", index=False)
